#### Aqui iniciei com o import do selenium
###### Mas antes foi preciso fazer a instalação das bibliotecas:
- pip install selenium
- pip install scrapy
- pip install ipython

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

##### Fiz a instalação do chromedriver e em seguida abro a página do Google

In [2]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:01<00:00, 4.16MB/s]


##### Em seguida, insiro a palavra chave para busca, e abro o Linkedin

In [3]:
driver.find_element('name', 'q').send_keys('vagas marketing')
driver.get("https://www.linkedin.com")

###### Ao abrir o Linkedin, não irei logar, portanto seleciono button para vagas

In [4]:
driver.find_elements(by=By.XPATH, value='/html/body/nav/ul/li[4]/a')[0].click()

###### Em seguida, limpo a label de localidade e a label de palavras-chaves para vagas, e insiro a que quero buscar

In [5]:
driver.find_element('name','keywords').clear()
driver.find_element('name','keywords').send_keys('Marketing E Publicidade')
driver.find_element('name','location').clear()
driver.find_element('name','location').send_keys('Brasil')

###### Seleciono o localidade 1 das opções que aparece ao preencher a label

In [6]:
driver.find_elements(by=By.XPATH, value='//*[@id="location-1"]')[0].click()

###### Aqui irei abrir os filtros para tipos de vaga, e selecionar o filtro Tempo integral e Estágio, por último seleciono o botão concluído

In [7]:
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/button')[0].click()

In [8]:
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/div/div/div[1]/label')[0].click()

In [9]:
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/div/div/div[5]/label')[0].click()

In [10]:
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/button')[0].click()

###### Pegando link das vagas

In [11]:
vagas_link = driver.find_elements(by=By.CLASS_NAME, value='base-card__full-link')
vagas_link=[link.get_attribute('href') for link in vagas_link]

print(vagas_link)
print()
print(len(vagas_link))

['https://br.linkedin.com/jobs/view/superior-em-publicidade-e-propaganda-ou-marketing-at-instituto-formar-3555791539?refId=MlRVYTHxaBo8MSlsMYfkrg%3D%3D&trackingId=WnPYyyL44hccb5%2BLP4UFSg%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/brand-designer-at-nelogica-3531121540?refId=MlRVYTHxaBo8MSlsMYfkrg%3D%3D&trackingId=0BKsP%2BkSHz59x9iPamcs3g%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/assistente-de-marketing-e-publicidade-at-equipe-cine-3521828190?refId=MlRVYTHxaBo8MSlsMYfkrg%3D%3D&trackingId=Nw0O0Rw8Ml7UlQQJz18bog%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/assistente-de-m%C3%ADdias-sociais-digitais-social-media-at-funda%C3%A7%C3%A3o-s%C3%A3o-paulo-3529648162?refId=MlRVYTHxaBo8MSlsMYfkrg%3D%3D&trackingId=ghToKTPerjg9qx1%2FLy9AYg%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://b

###### Pegando titulos das vagas

In [12]:
vagas_title = driver.find_elements(by=By.CLASS_NAME, value='base-search-card__title')
vagas_title=[titulo.text for titulo in vagas_title]
'''total_vagas = driver.find_elements(by=By.CLASS_NAME, value='results-context-header__job-count')[0]
total_vagas = total_vagas.text
total_vagas = int(float(total_vagas))*1000'''
'''v=[]
for i in vagas_title:
    #print(i.text)
    v.append(i.text)'''
print(vagas_title)
print()
print(len(vagas_title))

['SUPERIOR EM PUBLICIDADE E PROPAGANDA ou MARKETING', 'Brand Designer', 'Assistente de Marketing e Publicidade', 'Assistente de Mídias Sociais/Digitais ( Social Media )', 'Assistente de Publicidade e Marketing', 'Social Media Júnior', 'Analista de Publicidade', 'Analista de Social Media Jr.', 'Vendedor de Publicidade', 'Social Media', 'Assistente de Social Media', 'Assistente de Comunicação e Marketing', 'Assistente de design e social media', 'Assistente de Comunicação e Marketing - Florianópolis/SC', 'Assistente de Marketing', 'Marketing - Júnior Profissional - Trabalho Remoto', 'Social Media', 'Social Media - Freelance', 'Assistente de Comunicação e Marketing', 'Social media - Time de Criação', 'ESPECIALISTA DE MARKETING', 'Gerente de Marketing', 'Vendedor de publicidade', 'Assistente de Marketing (Social Media)', 'Social Media Júnior- Presencial']

25


###### Pegando nome das empresas das vagas

In [13]:
vagas_empresa = driver.find_elements(by=By.CLASS_NAME, value='hidden-nested-link')
vagas_empresa = [empresa.text for empresa in vagas_empresa]
'''for i in vagas_empresa:
    #print(i.text)
    emp.append(i.text)'''
print(vagas_empresa)
print()
print(len(vagas_empresa))

['Instituto Formar', 'Nelogica', 'Equipe Cine', 'Fundação São Paulo', 'Equipe Cine', 'M3 Social | MKT Digital', 'TrabalhaBrasil', 'UOL EdTech', 'IMMakers ADS', 'VISS Cabeleireiros Visagistas', 'Nestlé', 'Rede Verbita de Educação', 'It Stone Semi Joias', 'Transpocred – Cooperativa Ailos', 'Fourmi Publicidade', 'BairesDev', 'EstrelaBet', 'AKL Marketing Pet', 'Colégio Arnaldo', 'Like Design', 'Grupo 3corações', 'Copacol', 'Entrelinhas Conteúdo & Forma', 'Grupo Fiscal', 'RmaisH']

25


###### Pegando link das empresas das vagas

In [14]:
vagas_link_emp = driver.find_elements(by=By.CLASS_NAME, value='hidden-nested-link')
vagas_link_emp = [link_emp.get_attribute('href') for link_emp in vagas_link_emp]
'''for i in vagas_link_emp:
    #print(i.text)
    emp_link.append(i.get_attribute('href'))'''
print(vagas_link_emp)
print()
print(len(vagas_link_emp))

['https://br.linkedin.com/company/instituto-formar-es?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/nelogica?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/equipe-cine?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/funda%C3%A7%C3%A3o-s%C3%A3o-paulo?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/equipe-cine?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/agenciam3social?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/trabalhabrasil?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/ciatech-uoledtech?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/immakersads?trk=public_jobs_jserp-result_job-search-card-subtitle', 'https://br.linkedin.com/company/viss-cabeleireiro

In [27]:
def rasp(tipo, nivel):
    vagas_nivel = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span')
    for i in vagas_nivel:
        nivel.append(i.text)
    
    vagas_tipo = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[2]/span')
    for i in vagas_tipo:
        tipo.append(i.text)

In [28]:
vagas_tipo = []
vagas_nivel = []

for i in vagas_link:
    driver.get(i)
    rasp(vagas_nivel, vagas_tipo)

print(vagas_tipo)
print(len(vagas_tipo))
print(vagas_nivel)
print(len(vagas_nivel))

['Assistente', 'Assistente', 'Assistente', 'Assistente', 'Assistente', 'Não aplicável', 'Assistente', 'Pleno-sênior', 'Pleno-sênior', 'Não aplicável']
10
['Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral', 'Tempo integral']
10


In [35]:
num_seguidores_emp = []

'''for i in vagas_link_emp:
    driver.get(i)
    if 
    seguidores = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/section/div/div[2]/div[1]/h3/text()[2]')
    print(seguidores)'''

'for i in vagas_link_emp:\n    driver.get(i)\n    if \n    seguidores = driver.find_elements(by=By.XPATH, value=\'//*[@id="main-content"]/section[1]/section/div/div[2]/div[1]/h3/text()[2]\')\n    print(seguidores)'